In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import cv2


In [2]:
#creating 2 numpy arrays x containing images and y containing labels  
def preprocess(foldername,e,h,l): 
 
 x=np.zeros((e+h+l,144,144,3)) 
 y=np.zeros((e+h+l,1,3))   

 samples=['Early_Blight','Healthy','Late_Blight']  

 for j in samples:  
    
    for i,pic in enumerate(os.listdir('/Users/shrutisekhar/Desktop/icg project/'+foldername+'/'+j)):  
        img=cv2.imread(os.path.join('/Users/shrutisekhar/Desktop/icg project/'+foldername+'/'+j,pic),cv2.COLOR_BGR2RGB) 
        img=cv2.resize(img,(144,144),interpolation=cv2.INTER_AREA)  
        if img is None: 
            print(f'error in {pic}')
            continue 
        img=img/255 #scaling all pixel between 0 and 1
        if j=='Early_Blight': 
             x[i]=img
        if j=='Healthy': 
             x[i+e]=img 
        if j=='Late_Blight': 
             x[i+e+h]=img   
        
        values=np.zeros((1,3))
        if j=='Early_Blight': 
             values[0,0]=1 
             y[i]=values #0 for Early Blight  
             
        if j=='Healthy':  
             values[0,1]=1
             y[i+e]=values #1 for Healthy  
        if j=='Late_Blight':  
             values[0,2]=1
             y[i+e+h]=values #2 for Late Blight   
       
        
 return x,y

 


In [3]:
e=1303#no. of images in Early Blight
h=1303#no. of images in Healthy 
l=1132#no. of images in Late Blight 

x,y=preprocess('Training',e,h,l)
print(f'shape of x:{x.shape}') 
print(f'shape of y:{y.shape}')

shape of x:(3738, 144, 144, 3)
shape of y:(3738, 1, 3)


In [4]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Reshape,BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy 
from tensorflow.keras.callbacks import EarlyStopping


/Users/shrutisekhar/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
model=Sequential([  
                   
                   Conv2D(32,(3,3),activation='relu',strides=(2,2),kernel_initializer='he_normal',input_shape=(144,144,3),name='conv1') ,  
                   MaxPooling2D(pool_size=(2,2),name='mp1'),  

                   Conv2D(64,(3,3),activation='relu',strides=(1,1),kernel_initializer='he_normal',name='conv2') ,
                   MaxPooling2D(pool_size=(2,2),name='mp2'),  
                   

                   Conv2D(128,(3,3),activation='relu',strides=(1,1),kernel_initializer='he_normal',name='conv3'),   
                   BatchNormalization(),
                   MaxPooling2D(pool_size=(2,2),name='mp3'), 

                   Flatten(name='flatten'), 
                   
                   Dense(128,activation='relu',name='dense1',kernel_initializer='he_normal') , 
                   Dropout(0.5,name='dropout4') , 
                   Dense(64,activation='relu',name='dense2',kernel_initializer='he_normal') ,
                   Dense(3,activation='softmax',name='output'),  

                   Reshape((1,3),name='reshape_output') 
                   
   ])  
model.compile(optimizer='rmsprop',loss=CategoricalCrossentropy,metrics=['accuracy']) 

 

/Users/shrutisekhar/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 71, 71, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mp1 (MaxPooling2D)              │ (None, 35, 35, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 33, 33, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mp2 (MaxPooling2D)              │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mp3 (MaxPooling2D)              │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout4 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │           195 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_output (Reshape)        │ (None, 1, 3)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 905,155 (3.45 MB)

 Trainable params: 904,899 (3.45 MB)

 Non-trainable params: 256 (1.00 KB)

In [7]:
early_stopping=EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True) 


In [8]:
from sklearn.model_selection import train_test_split 

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [9]:
history=model.fit(x_train,y_train,epochs=150,validation_data=(x_val,y_val))

Epoch 1/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 45ms/step - accuracy: 0.3878 - loss: 1.8124 - val_accuracy: 0.4225 - val_loss: 1.0481
Epoch 2/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.3578 - loss: 0.9641 - val_accuracy: 0.3222 - val_loss: 1.0447
Epoch 3/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4360 - loss: 0.8700 - val_accuracy: 0.3275 - val_loss: 1.4529
Epoch 4/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.4531 - loss: 0.8452 - val_accuracy: 0.4626 - val_loss: 1.2721
Epoch 5/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 49ms/step - accuracy: 0.4801 - loss: 0.8465 - val_accuracy: 0.3476 - val_loss: 1.1110
Epoch 6/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.5401 - loss: 0.7482 - val_accuracy: 0.3008 - val_loss: 2.2641
Epoch 7/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.5385 - loss: 0.7373 - val_accuracy: 0.5281 - val_loss: 0.8165
Epoch 8/150
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.6117 - loss: 0.6164 - val_accuracy: 0.

In [10]:
x_test,y_test=preprocess('Testing',162,102,141) 
y_pred_prob=model.predict(x_test)

y_pred=np.argmax(np.squeeze(y_pred_prob),axis=1) 
y_true=np.argmax(np.squeeze(y_test),axis=1)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [11]:
print(y_pred_prob)

[[[1.0000000e+00 1.4151143e-29 3.4127770e-10]]

 [[8.2145047e-01 1.4620878e-01 3.2340765e-02]]

 [[1.0000000e+00 0.0000000e+00 2.5372017e-13]]

 ...

 [[9.0419938e-04 8.4620588e-08 9.9909568e-01]]

 [[4.2981929e-15 5.1785172e-17 1.0000000e+00]]

 [[0.0000000e+00 0.0000000e+00 1.0000000e+00]]]


In [12]:
print(y_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 1 1 2 1 0 1 1 0 0 1 0 0 1 1 1 0 1 0 1 0 1 1
 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 0 1 1
 0 1 1 1 1 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 0 2 2 2 1 2 2 2 2 2 2 2 0 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2
 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 0 0 2 2 2]


In [13]:
print(y_true)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [14]:
 
print(np.shape(y_true) )
print(np.shape(y_pred))

(405,)
(405,)


In [15]:
#f1 score 
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average = 'weighted')
print(f"F1 Score: {f1}")

F1 Score: 0.9105833846969043
